##**07. 문서 군집화 소개와 실습(Opinion Review 데이터 세트)**
**[문서 군집화 개념]**

:문서 군집화는 비슷한 텍스트 구성의 문서를 군집화하는 것. 학습 데이터 세트가 필요없는 비지도 학습.

**[Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기]**



In [1]:
#여러개의 파일을 DataFrame으로 로딩
from nltk.stem import WordNetLemmatizer
import nltk
import string

# 단어 원형 추출 함수
lemmar = WordNetLemmatizer()
def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# 특수 문자 사전 생성: {33: None ...}
# ord(): 아스키 코드 생성
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)


def LemNormalize(text):
    # 텍스트 소문자 변경 후 특수 문자 제거
    text_new = text.lower().translate(remove_punct_dict)

    # 단어 토큰화
    word_tokens = nltk.word_tokenize(text_new)

    # 단어 원형 추출
    return LemTokens(word_tokens)

In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
import pandas as pd
import glob,os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

path = r'/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1.0/topics'
all_files = glob.glob(os.path.join(path, "*.data"))
filename_list = []
opinion_text = []

for file_ in all_files:
    df = pd.read_table(file_, index_col = None, header=0, encoding='latin1')
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]
    filename_list.append(filename)
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...
1,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...
2,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...
3,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...
4,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...


문서를 TF-IDF 형태로 피처 벡터화. TfidVectorizer은 tokenizer인자에 커스텀 어근 변환 함수를 적용해 어근 변환을 수행할 수 있음.


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words = 'english',
                             ngram_range=(1,2), min_df=0.05, max_df = 0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:406: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [5]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [6]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
1,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
2,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
3,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
4,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1


In [8]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
7,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
13,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
14,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
20,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
24,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,headphone jack i got a clear case for it an...,0
27,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
34,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
39,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0


In [9]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
2,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
43,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
40,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
38,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
37,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
36,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
35,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
45,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
33,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1
23,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,1


In [10]:
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
6,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
8,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
12,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
16,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
30,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
31,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
32,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2
47,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,2


In [11]:
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
5,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
10,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
18,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
41,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
44,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
48,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3
50,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,3


In [12]:
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
3,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
19,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
21,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
22,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
25,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
26,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
28,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
42,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4
49,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,4


군집의 개수를 5개로 하니 군집 개수가 약간 많아 세분화되어 군집화된 경향이 있음. 중심 개수를 5개에서 3개로 낮춰서 3개 그룹으로 군집화한 뒤 결과를 확인.

In [13]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by = 'cluster_label')

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
20,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
21,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
22,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
24,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,headphone jack i got a clear case for it an...,0
49,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
26,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
19,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
27,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0
34,/content/drive/MyDrive/2024_2학기/ESAA OB/필...,...,0


**[군집별 핵심 단어 추출하기]**

각 군집에 속한 문서는 핵심단어를 주축으로 군집화돼 있을 것.

KMeans 객체는 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해 있는지 clusters_centers_라는 속성으로 제공. 각 배열내의 값은 개별 군집 내의 상대 위치를 숫자 값으로 표현한 일종의 좌표 값.

In [14]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape:', cluster_centers.shape)
print(cluster_centers)

cluster_centers shape: (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


각 행의 배열값은 각 군집 내의 피처의 위치가 개별 중심과 얼마나 가까운가를 상대 값으로 나타낸 것. 0에서 1까지의 값을 가질 수 있으며, 1에 가까울수록 중심과 가까운값을 의미.

cluster_centers_속성값을 이용해 각 군집별 핵심 단어를 찾아봄.

In [23]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
  cluster_details={}

  centroid_feature_ordered_ind=cluster_model.cluster_centers_.argsort()[:, ::-1]

  # 개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
  for cluster_num in range(clusters_num):
    # 개별 군집별 정보를 담을 데이터 초기화
    cluster_details[cluster_num]={}
    cluster_details[cluster_num]['cluster']=cluster_num

    # cluster_centers_argsort()[:, ::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
    top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
    top_features=[feature_names[ind] for ind in top_feature_indexes]

    # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함.
    top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

    # cluster_details 딕셔너리 객체에 개별 군집별 핵심단어와 중심위치 상댓값, 해당 파일명 입력
    cluster_details[cluster_num]['top_features']=top_features
    cluster_details[cluster_num]['top_features_value']=top_feature_values
    filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
    filenames=filenames.values.tolist()

    cluster_details[cluster_num]['filenames']=filenames

  return cluster_details

In [24]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('####### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명 : ', cluster_detail['filenames'][:7])
    print('=================================================')

In [25]:
feature_names = tfidf_vect.get_feature_names_out()

cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,
                                    feature_names=feature_names, clusters_num=3, top_n_features=10)

print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명 :  ['/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 :  ['/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2학기/ESAA OB/필사 과제/OpinosisDataset1', '/content/drive/MyDrive/2024_2하